#### Imports

In [ ]:
import os
from pathlib import Path
import copy
from typing import TypeAlias
import numpy as np
import numpy.typing as npt
import py4spice as spi
from py4spice import globals_types

#### Constants and directories

In [ ]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_book_sim/circuits/simple_rc")

# where to put results
RESULTS_PATH = PROJ_PATH / "sim_results"

# netlist filenames
NETLISTS_PATH = PROJ_PATH / "netlists"
TOP_FILENAME = NETLISTS_PATH / "top.cir"
TITLE_FILENAME = NETLISTS_PATH / "title.cir"
DUT_FILENAME = NETLISTS_PATH / "dut.cir"
LOAD_FILENAME = NETLISTS_PATH / "load.cir"
STIMULUS_FILENAME = NETLISTS_PATH / "stimulus.cir"
SUPPLIES_FILENAME = NETLISTS_PATH / "supplies.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

#### Create netlists objects. We'll combine them after "control" netlist is constructed.

In [ ]:
title = spi.Netlist(NETLISTS_PATH / TITLE_FILENAME)
dut = spi.Netlist(NETLISTS_PATH / DUT_FILENAME)
load = spi.Netlist(NETLISTS_PATH / LOAD_FILENAME)
supplies = spi.Netlist(NETLISTS_PATH / SUPPLIES_FILENAME)
stimulus = spi.Netlist(NETLISTS_PATH / STIMULUS_FILENAME)

#### Create signal vectors

In [ ]:
VEC_INTEREST = spi.Vectors("in mid")

#### Define analyses

In [ ]:
list_of_analyses: list[spi.Analyses] = []

# 1st analysis: operating point
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_INTEREST, RESULTS_PATH)
list_of_analyses.append(op1)

# 2nd analysis: transient with time-end = 20m
tr_cmd = "tran 1e-3 20e-3"
tr1 = spi.Analyses("tr1", "tran", tr_cmd, VEC_INTEREST, RESULTS_PATH)
list_of_analyses.append(tr1)

#### Create control section

In [ ]:
my_control = spi.Control()  # create 'my_control' object
my_control.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses:  # statements for all analyses
    my_control.insert_lines(analysis.lines_for_cntl())
spi.print_section("Control File", my_control)  # print out contents
my_control.content_to_file(CONTROL_FILENAME)  # creat the actual file
control = spi.Netlist(NETLISTS_PATH / CONTROL_FILENAME)

#### Simulate

In [ ]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = title + dut + load + supplies + stimulus + control
top.writefile(TOP_FILENAME)
spi.print_section("top netlist", top)

sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run()

#### Simulation results to numpy

In [ ]:
sim_results = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

#### First simulation results: operating point

In [ ]:
# 1st simulation was operating point
spi.print_section("Operating Point Results", sim_results[0].print_table())


#### Second simulation results: transient

In [ ]:
# x-axis and signals in list of numpys
plot_data = sim_results[1].x_axis_and_sigs(VEC_INTEREST.list_out)

# list of signals we want on plot
y_names = VEC_INTEREST.list_out

# Create plot with matplotlib
plt_tr = spi.Plot("tr_plt", plot_data, y_names, RESULTS_PATH)
plt_tr.set_title("transient results")
plt_tr.define_axes(("time", "sec", "linear"), ("voltage", "V", "linear"))